In [ ]:
%pip install -qr requirements.txt # install

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [ ]:
import torch
import utils
display = utils.notebook_init()  # checks

# 1. Detect

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/LNwODJXcvt4'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://user-images.githubusercontent.com/26833433/127574988-6a558aa1-d268-44b9-bf6b-62d4c605cc72.jpg" width="600">

# 2. Validate
Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag.

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

# 3. Train

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

# Train Pretrained

In [ ]:
!python train.py --data coco128.yaml --weights 'yolov5s.pt' --cfg models/yolov5s.yaml --img 416 --epochs 3

# Train QuantModel, not pretrained

In [ ]:
!python train.py --data coco128.yaml --weights '' --cfg models/yolov5s_quant.yaml --img 416 --epochs 1 --classes 80

In [ ]:
ckpt = {}
ckpt['model'] = model.state_dict()
torch.save(ckpt,"experiment_models/yolov5s.pt")

# Train Quantized (QAT)

In [ ]:
# please do not forget to add the --cfg for quant models so the code does not break
!python train.py --img 416 --batch 16 --epochs 3 --data coco128.yaml --cfg models/yolov5s_quant.yaml --weights experiment_models/yolov5s_quant.pt 

# Train PostQuantization (PTQ)

In [ ]:
# please do not forget to add the --cfg for quant models so the code does not break
!python train.py --img 416 --batch 16 --epochs 9 --data coco128.yaml --cfg models/yolov5s_quant.yaml --weights yolov5s.pt --device 0

# Inference Pretrained Model

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model_pre = torch.hub.load('.',
                       'custom',
                       'experiment_models/yolov5s.pt',
                       source='local',
                       classes = 80,
                       cfg = "models/yolov5s.yaml",
                       force_reload=True
                      )

im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model_pre(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

In [ ]:
for key,value in model_pre.state_dict().items():
    print(key,value)

# Inference Quantized Model (QAT)

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('.',
                       'custom',
                       'experiment_models/yolov5s_quant_pre.pt',
                       source='local',
                       classes = 80,
                       cfg = "models/yolov5s_quant.yaml",
                       force_reload=True
                      )

im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

In [ ]:
results = model(im)  # inference
results.print()

In [ ]:
for key,value in model.state_dict().items():
    print(key,value)

In [ ]:
model.load_state_dict(model_pre.state_dict(), strict = False)
model.cuda()

# Inference Quantized Model (PTQ)

In [1]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('.',
                       'custom',
                       'experiment_models/yolov5s_quant_pre.pt',
                       source='local',
                       classes = 80,
                       cfg = "models/yolov5s_quant.yaml",
                       force_reload=True
                      )

im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TypeError: str expected, not NoneType

In [ ]:
df = results.pandas()
df.xyxy

# Copying weights for PTQ

In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model_quant = torch.hub.load('.',
                       'custom',
                       'experiment_models/yolov5s_quant.pt',
                       source='local',
                       classes = 80,
                       cfg = "models/yolov5s_quant.yaml",
                       force_reload=False
                      )

model = torch.hub.load('.',
                       'custom',
                       'experiment_models/yolov5s.pt',
                       source='local',
                       classes = 80,
                       cfg = "models/yolov5s.yaml",
                       force_reload=False
                      )


In [ ]:
model.model.model.model[0].conv.weight.data

In [ ]:
model_quant_state = model_quant.state_dict()
model_state = model.state_dict()

for key,value in model_state.items():
    if key in model_quant_state:
        print(key)
        model_quant_state[key] = value
    else:
        print("NotFound",key)
        
model_quant.load_state_dict(model_quant_state)

In [ ]:
results = model_quant(im)
results

In [ ]:
model_quant.model.model.model[0].conv.weight.data

In [ ]:
model_quant.load_state_dict(model.state_dict(),strict = False)